# 第十四讲：因子分析的EM算法、主成分分析

先来回顾一下上一讲推导出的结论，对于随机变量$x=\begin{bmatrix}x_1\\x_2\end{bmatrix},\ x\sim\mathcal N(\mu,\varSigma)$，其中$\mu=\begin{bmatrix}\mu_1\\\mu_2\end{bmatrix},\ \varSigma=\begin{bmatrix}\varSigma_{11}&\varSigma_{12}\\\varSigma_{21}&\varSigma_{22}\end{bmatrix}$，则有：

* $x_1$的边缘分布$p(x_1)$为：

$$
x\sim\mathcal N\left(\mu_1,\varSigma_{11}\right)
$$

* 在$x_2$条件下$x_1$的概率$p(x_1\mid x_2)$为：

$$
\begin{align}
x_1\mid x_2&\sim\mathcal N\left(\mu_{1\mid2},\varSigma_{1\mid2}\right)\\
\mu_{1\mid2}&=\mu_1+\varSigma_{12}\varSigma_{22}^{-1}(x_2-\mu_2)\\
\varSigma_{1\mid2}&=\varSigma_{11}-\varSigma_{12}\varSigma_{22}^{-1}\varSigma_{21}
\end{align}
$$

## 4. 因子分析模型的EM算法

E步骤的推导比较简单，我们需要计算$Q_i\left(z^{(i)}\right)=p\left(z^{(i)}\mid x^{(i)};\mu,\varLambda,\varPsi\right)$，将[上一讲](chapter13.ipynb)中的$(1),(2)$式带入$(3)$式，计算条件概率$z^{(i)}\mid x^{(i)}$：

$$
\begin{align}
z^{(i)}\mid x^{(i)};\mu,\varLambda,\varPsi&\sim\mathcal N\left(\mu_{z^{(i)}\mid x^{(i)}},\varSigma_{z^{(i)}\mid x^{(i)}}\right)\\
\mu_{z^{(i)}\mid x^{(i)}}&=\varLambda^T\left(\varLambda\varLambda^T+\varPsi\right)^{-1}\left(x^{(i)}-\mu\right)\\
\varSigma_{z^{(i)}\mid x^{(i)}}&=I-\varLambda^T\left(\varLambda\varLambda^T+\varPsi\right)^{-1}\varLambda
\end{align}
$$

将$\mu_{z^{(i)}\mid x^{(i)}},\varSigma_{z^{(i)}\mid x^{(i)}}$带入高斯分布$Q_i\left(z^{(i)}\right)$得：

$$
Q_i\left(z^{(i)}\right)=\frac{1}{\sqrt{(2\pi)^k\left\lvert\varSigma_{z^{(i)}\mid x^{(i)}}\right\rvert}}\exp\left(-\frac{1}{2}\left(z^{(i)}-\mu_{z^{(i)}\mid x^{(i)}}\right)^T\varSigma_{z^{(i)}\mid x^{(i)}}^{-1}\left(z^{(i)}-\mu_{z^{(i)}\mid x^{(i)}}\right)\right)
$$

再来看M步骤，我们需要最大化下式关于参数$\mu,\varLambda,\varPsi$的函数：

$$
\sum_{i=1}^m\int_{z^{(i)}}Q_i\log\frac{p\left(x^{(i)},z^{(i)};\mu,\varLambda,\varPsi\right)}{Q_i\left(z^{(i)}\right)}\mathrm dz^{(i)}\tag{4}
$$

我们这里只写出关于优化$\varLambda$的推导，其他参数的推导过程类似，$(4)$式可化简为：

$$
\begin{align}
\sum_{i=1}^m\int_{z^{(i)}}Q_i\left[\log p\left(x^{(i)}\mid z^{(i)};\mu,\varLambda,\varPsi\right)+\log p\left(z^{(i)}\right)-\log Q_i\left(z^{(i)}\right)\right]\mathrm dz^{(i)}\tag{4}\\
=\sum_{i=1}^m\operatorname*{E}_{z^{(i)}\sim Q_i}\left[\log p\left(x^{(i)}\mid z^{(i)};\mu,\varLambda,\varPsi\right)+\log p\left(z^{(i)}\right)-\log Q_i\left(z^{(i)}\right)\right]\tag{5}
\end{align}
$$

$\displaystyle\operatorname*{E}_{z^{(i)}\sim Q_i}$指的服从分布$Q_i$的随机变量$z^{(i)}$的期望。后面的推导中，我们会在表达不产生歧义时省略这个下标。继续化简，删掉式中与$\varLambda$无关的项，则需要最大化的目标为：

$$
\begin{align}
\sum_{i=1}^m&\mathrm E\left[\log p\left(x^{(i)}\mid z^{(i)};\mu,\varLambda,\varPsi\right)\right]\\
=&\sum_{i=1}\mathrm E\left[\log\frac{1}{\sqrt{(2\pi)^n\lvert\varPsi\rvert}}\exp\left(-\frac{1}{2}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)^T\varPsi^{-1}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)\right)\right]\\
=&\sum_{i=1}^m\mathrm E\left[-\frac{1}{2}\log\lvert\varPsi\rvert-\frac{n}{2}\log(2\pi)-\frac{1}{2}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)^T\varPsi^{-1}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)\right]
\end{align}
$$

最大化上面这个关于$\varLambda$的函数，只有最后一项与$\varLambda$有关，于是对$\varLambda$求偏导，并使用[第二讲](chapter02.ipynb)中提到的性质：$\forall a\in\mathbb R,\mathrm{tr}a=a$、$\mathrm{tr}AB=\mathrm{tr}BA$以及$\nabla_\varLambda\mathrm{tr}ABA^TC=CAB+C^TAB$这三条性质得到（$\varLambda$是对角矩阵）：

$$
\begin{align}
\nabla_\varLambda\sum_{i=1}^m-&\mathrm E\left[\frac{1}{2}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)^T\varPsi^{-1}\left(x^{(i)}-\mu-\varLambda z^{(i)}\right)\right]\\
=\sum_{i=1}^m&\nabla_\varLambda\mathrm E\left[-\mathrm{tr}\left(\frac{1}{2}\left(z^{(i)}\right)^T\varLambda^T\Psi^{-1}\varLambda z^{(i)}\right)+\mathrm{tr}\left(\left(z^{(i)}\right)^T\varLambda^T\Psi^{-1}\left(x^{(i)}-\mu\right)\right)\right]\\
=\sum_{i=1}^m&\nabla_\varLambda\mathrm E\left[-\mathrm{tr}\frac{1}{2}\varLambda^T\varPsi^{-1}\varLambda z^{(i)}\left(z^{(i)}\right)^T+\mathrm{tr}\varLambda^T\varPsi^{-1}\left(x^{(i)}-\mu\right)\left(z^{(i)}\right)^T\right]\\
=\sum_{i=1}^m&\mathrm E\left[-\varPsi^{-1}\varLambda z^{(i)}\left(z^{(i)}\right)^T+\varPsi^{-1}\left(x^{(i)}-\mu\right)\left(z^{(i)}\right)^T\right]
\end{align}
$$

将上面的结果置为零并化简得到：

$$
\sum_{i=1}^m\varLambda\operatorname*{E}_{z^{(i)}\sim Q_i}\left[z^{(i)}\left(z^{(i)}\right)^T\right]=\sum_{i=1}^m\left(x^{(i)}-\mu\right)\operatorname*{E}_{z^{(i)}\sim Q_i}\left[\left(z^{(i)}\right)^T\right]
$$

最后，解出$\varLambda$得到：

$$
\varLambda=\left(\sum_{i=1}^m\left(x^{(i)}-\mu\right)\operatorname*{E}_{z^{(i)}\sim Q_i}\left[\left(z^{(i)}\right)^T\right]\right)\left(\sum_{i=1}^m\operatorname*{E}_{z^{(i)}\sim Q_i}\left[z^{(i)}\left(z^{(i)}\right)^T\right]\right)^{-1}\tag{7}
$$

有趣的是，这个表达式和我们从最小二乘回归得出的正规方程组$\theta^T=\left(y^TX\right)\left(X^TX\right)^{-1}$有着相似之处，可以看出$x$是关于$z$（加上噪音项）的线性函数。在E步骤中猜测$z$之后，我们尝试估计$x$与$z$之间的线性关系$\varLambda$。这个相似点到没什么，但应该注意两个算法之间的重要不同——最小二乘法会对$z$进行“最佳猜测”，我们在后面会看到这个不同。

要完成M步骤的参数$\varLambda$更新，我们需要把$(7)$式计算出来。由$Q_i$的定义可知，它是一个以$\mu_{z^{(i)}\mid x^{(i)}}$为期望，$\varSigma_{z^{(i)}\mid x^{(i)}}$为方差的高斯分布，则有：

$$
\begin{align}
\operatorname*{E}_{z^{(i)}\sim Q_i}\left[\left(z^{(i)}\right)^T\right]&=\mu_{z^{(i)}\mid x^{(i)}}^T\\
\operatorname*{E}_{z^{(i)}\sim Q_i}\left[z^{(i)}\left(z^{(i)}\right)^T\right]&=\mu_{z^{(i)}\mid x^{(i)}}\mu_{z^{(i)}\mid x^{(i)}}^T+\varSigma_{z^{(i)}\mid x^{(i)}}
\end{align}
$$

上面的第二个式子可以从协方差的定义推出，对于随机变量$Y$，$\mathrm{Cov}(Y)=\mathrm E\left[YY^T\right]-\mathrm E[Y]\mathrm E[Y]^T$，$\mathrm E\left[YY^T\right]=\mathrm E[Y]\mathrm E[Y]^T+\mathrm{Cov}(Y)$，带回$(7)$式即可得到$\varLambda$的更新规则：

$$
\varLambda=\left(\sum_{i=1}^m\left(x^{(i)}-\mu\right)\mu_{z^{(i)}\mid x^{(i)}}^T\right)\left(\sum_{i=1}^m\mu_{z^{(i)}\mid x^{(i)}}\mu_{z^{(i)}\mid x^{(i)}}^T+\varSigma_{z^{(i)}\mid x^{(i)}}\right)^{-1}\tag{8}
$$

我们一定要知道等式右边$\varSigma_{z^{(i)}\mid x^{(i)}}$的意义，这是在$x^{(i)}$条件下$z^{(i)}$的后验分布$p\left(z^{(i)}\mid x^{(i)}\right)$的协方差，而M步骤必须将这个关于$z^{(i)}$的不确定项考虑在内。一个在EM推导中常见的错误就是认为在E步骤中只需要计算潜在随机变量的期望$\mathrm E[z]$，之后用它代替M步骤优化中出现的所有$z$就可以完成了。虽然这样确实对简单的模型，比如混合高斯分布有效，但在我们对因子分析模型的推导中，除了$\mathrm E[z]$之外还需要$\mathrm E\left[zz^T\right]$，而且我们在上面确实看到了$\mathrm E[z]$与$\mathrm E\left[zz^T\right]$之间相差了$\varSigma{z\mid x}$项。因此，M步骤必须将$z$变量在后验分布$p\left(z^{(i)}\mid x^{(i)}\right)$上的协方差考虑在内。

最后，我们给出M步骤中其他参数$\mu,\varPsi$的更新规则，推导这些参数并不难，首先是参数$\mu$的更新规则：

$$
\mu=\frac{1}{m}\sum_{i=1}^mx^{(i)}
$$

参数$\mu$并不像其他变量那样在更新中变化（也就是说和参数$\varLambda$不同，式子右边并不依赖$Q_i\left(z^{(i)}\right)=p\left(z^{(i)}\mid x^{(i)};\mu,\varLambda,\varPsi\right)$，也就不依赖其他参数了），这个参数只需要计算一次，并且随着算法的执行，这个参数也不需要再做更新。类似的，对角矩阵$\varPsi$的更新规则为：

$$
\varPhi=\frac{1}{m}\sum_{i=1}^mx^{(i)}\left(x^{(i)}\right)^T-x^{(i)}\mu_{z^{(i)}\mid x^{(i)}}\varLambda^T-\varLambda\mu_{z^{(i)}\mid x^{(i)}}\left(x^{(i)}\right)^T+\varLambda\left(\mu_{z^{(i)}\mid x^{(i)}}\mu_{z^{(i)}\mid x^{(i)}}^T+\varSigma_{z^{(i)}\mid x^{(i)}}\right)\varLambda^T
$$

再令$\varPsi_{ii}=\varPhi_{ii}$即可得到对角矩阵$\varPsi$（也就是让$\varPsi$只取$\varPhi$的对角线元素）。

\left(\right)
\begin{align}\end{align}
\begin{bmatrix}\end{bmatrix}